<a href="https://colab.research.google.com/github/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [15]:
#Upload file in sample_data folder
#https://github.com/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/Lab_5/jester-data-1.csv
df = pd.read_csv("jester-data-1.csv", header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [16]:
#Helper function
arr = df.values
print(arr.shape)

(24983, 101)


In [17]:
# shorthand for np.asarray(condition).nonzero().
rated = np.where(arr!=99)
print(rated)
print(len(rated[0]), rated[1].shape)

(array([    0,     0,     0, ..., 24982, 24982, 24982]), array([ 0,  1,  2, ..., 70, 71, 87]))
1835357 (1835357,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [19]:
new_arr, idx = replace(arr, 0.1)
print(new_arr)

[[ 74.    99.     8.79 ...  99.    99.    99.  ]
 [100.     4.08  -0.29 ...   0.34  99.     1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]


In [20]:
print(arr[idx[0][0], idx[1][0]])
print(new_arr[idx[0][0], idx[1][0]])

-1.99
99.0


***Latent Features***

In [21]:
n_latent_factors = 2

user_ratings = df.values
print(user_ratings)
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
print(latent_user_preferences)
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
print(latent_item_features)

[[ 74.    -7.82   8.79 ...  99.    99.    99.  ]
 [100.     4.08  -0.29 ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]
[[0.89519424 0.80801041]
 [0.46072765 0.50525453]
 [0.51536446 0.15559621]
 ...
 [0.35951595 0.76321017]
 [0.74529941 0.57805578]
 [0.214441   0.95984477]]
[[0.73059887 0.3458005 ]
 [0.89237666 0.59278286]
 [0.4425242  0.77231431]
 [0.56160985 0.86630339]
 [0.7486216  0.54724466]
 [0.4202979  0.06211873]
 [0.94575366 0.30094505]
 [0.09291056 0.71645495]
 [0.74910047 0.22243797]
 [0.73438872 0.4003098 ]
 [0.48079197 0.75306944]
 [0.19542856 0.10626763]
 [0.47044662 0.46197802]
 [0.34504203 0.19368376]
 [0.58048941 0.22170367]
 [0.28524236 0.6309388 ]
 [0.38298182 0.25387985]
 [0.47519133 0.17318108]
 [0.44399658 0.324972  ]
 [0.1450725  0.46095422]
 [0.67531817 0.90477352]
 [0.37491963 0.3225

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [28]:
#sgd()
#sgd(100)
sgd(20) 

nan


KeyboardInterrupt: ignored

In [0]:
pred = latent_user_preferences.dot(latent_item_features.T)
print(pred)

In [0]:
values = [zip(user_ratings[i], pred[i]) for i in range(pred.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
# the first number is the orginal value from the jester-ata-1.csv dataset, 
# the second number is the predicted value
print(comparison_data)

In [0]:
#Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
error=(pred[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse = sum(error)/len(error)
mse

Changing hyper-parameters (learning rate, number of iterations of SVD, number of latent factors)

In [0]:
n_latent_factors2 = 3

user_ratings = df.values
user_ratings
# Initialise as random values
latent_user_preferences2 = np.random.random((user_ratings.shape[0], n_latent_factors2))
latent_item_features2 = np.random.random((user_ratings.shape[1], n_latent_factors2))

In [0]:
def predict_rating2(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference2 = latent_user_preferences2[user_id]
    item_preference2 = latent_item_features2[item_id]
    return user_preference2.dot(item_preference2)


def train2(user_id, item_id, rating, alpha=0.001):
    
    #print item_id
    prediction_rating2 = predict_rating2(user_id, item_id)
    err =  prediction_rating2 - rating
    #print err
    user_pref_values2 = latent_user_preferences2[user_id][:]
    latent_user_preferences2[user_id] -= alpha * err * latent_item_features2[item_id]
    latent_item_features2[item_id] -= alpha * err * user_pref_values2
    return err
    

def sgd2(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences2.shape[0]):
            for item_id in range(latent_item_features2.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train2(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
sgd2(30)

In [0]:
pred2 = latent_user_preferences2.dot(latent_item_features2.T)
pred2

In [0]:
values2 = [zip(user_ratings[i], pred2[i]) for i in range(pred2.shape[0])]
comparison_data2 = pd.DataFrame(values2)
comparison_data2.columns = df.columns

In [0]:
comparison_data2

In [0]:
error2=(pred2[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse2 = sum(error2)/len(error2)
mse2